In [2]:
# pip install pandas_datareader  yfinance numpy torch matplotlib optuna scikit-learn
from pandas_datareader import data as pdr
import yfinance as yfin
import numpy as np
import utils
from torch.nn import DataParallel
from utils import merge_fred, get_table_by_id_yahoo
from utils import get_table_by_id_fred, get_table_by_id, add_target, split_shuffle
yfin.pdr_override()


In [3]:
start ='1960-01-01'
end = '2023-08-01'

In [4]:
import pandas as pd
from utils import add_target, get_table_by_id_yahoo, merge_fred, remove_nan
import os

future_period = 128



# check if the data file exists
if os.path.isfile(f'data/df_X_GSPC_start_{start}_end_{end}.csv'):
  df_X = pd.read_csv(f'data/df_X_GSPC_start_{start}_end_{end}.csv', index_col=0)
  df_y = pd.read_csv(f'data/df_y_GSPC_start_{start}_end_{end}.csv', index_col=0)
  df = pd.read_csv(f'data/df_GSPC_start_{start}_end_{end}.csv', index_col=0)
else:
  df, column_names = get_table_by_id_yahoo('^GSPC', 10, open=False, start=start, end=end)
  df, target_name = add_target(df, '^GSPC', future_period=future_period)

  df = df[column_names + [target_name]].copy()
  # Extract the day of the week (Monday=0, Sunday=6)
  # df['day_of_week'] = df.index.dayofweek

  # # Extract the day of the month
  # df['day_of_month'] = df.index.day

  # # Extract the day of the year
  # df['day_of_year'] = df.index.dayofyear

  #df_X = df[column_names+['day_of_week', 'day_of_month', 'day_of_year']].copy()
  #price_columns = [col for col in column_names if 'close' in col]
  
  df = merge_fred(df, 'M2SL', 7, start, end, 4, 2, if_log=True)
  df = merge_fred(df, 'UNRATE', 6, start, end, 1, 5, if_log=False)
  # df_X = merge_fred(df_X, 'GDP', 4, 3, start, end, if_log=True)

  df = merge_fred(df, 'FEDFUNDS', 6, start, end, 1, 5, if_log=False)
  df_X = df.drop(columns=['LAST_METRIC_DATE', target_name]).copy()

  df_y = df[[target_name]].copy()
  
  # write to csv
  df_X.to_csv(f'data/df_X_GSPC_start_{start}_end_{end}.csv')
  df_y.to_csv(f'data/df_y_GSPC_start_{start}_end_{end}.csv')
  df.to_csv(f'data/df_GSPC_start_{start}_end_{end}.csv')



In [5]:
df_X.columns

Index(['log_adj_close_diff_^GSPC_1', 'log_volume_diff_^GSPC_1',
       'log_adj_close_diff_^GSPC_2', 'log_volume_diff_^GSPC_2',
       'log_adj_close_diff_^GSPC_4', 'log_volume_diff_^GSPC_4',
       'log_adj_close_diff_^GSPC_8', 'log_volume_diff_^GSPC_8',
       'log_adj_close_diff_^GSPC_16', 'log_volume_diff_^GSPC_16',
       'log_adj_close_diff_^GSPC_32', 'log_volume_diff_^GSPC_32',
       'log_adj_close_diff_^GSPC_64', 'log_volume_diff_^GSPC_64',
       'log_adj_close_diff_^GSPC_128', 'log_volume_diff_^GSPC_128',
       'log_adj_close_diff_^GSPC_256', 'log_volume_diff_^GSPC_256',
       'log_adj_close_diff_^GSPC_512', 'log_volume_diff_^GSPC_512',
       'log_M2SL_diff_1', 'log_M2SL_diff_2', 'log_M2SL_diff_4',
       'log_M2SL_diff_8', 'log_M2SL_diff_16', 'log_M2SL_diff_32',
       'log_M2SL_diff_64', 'UNRATE_diff_1', 'UNRATE_diff_2', 'UNRATE_diff_4',
       'UNRATE_diff_8', 'UNRATE_diff_16', 'UNRATE_diff_32', 'FEDFUNDS_diff_1',
       'FEDFUNDS_diff_2', 'FEDFUNDS_diff_4', 'FEDFUNDS_

In [6]:


df, df_X, df_y, df_top, df_top_X = remove_nan(df, df_X, df_y, type='top')
df, df_X, df_y, df_tail, df_tail_X = remove_nan(df, df_X, df_y, type='bottom')

print(df_X.shape)
print(df_y.shape)
print(df.shape)
print(df_tail.shape)

Nan rows from top: 1341
Nan rows from bottom:  129
(14496, 39)
(14496, 1)
(14496, 41)
(129, 41)


In [7]:
df_X.columns

Index(['log_adj_close_diff_^GSPC_1', 'log_volume_diff_^GSPC_1',
       'log_adj_close_diff_^GSPC_2', 'log_volume_diff_^GSPC_2',
       'log_adj_close_diff_^GSPC_4', 'log_volume_diff_^GSPC_4',
       'log_adj_close_diff_^GSPC_8', 'log_volume_diff_^GSPC_8',
       'log_adj_close_diff_^GSPC_16', 'log_volume_diff_^GSPC_16',
       'log_adj_close_diff_^GSPC_32', 'log_volume_diff_^GSPC_32',
       'log_adj_close_diff_^GSPC_64', 'log_volume_diff_^GSPC_64',
       'log_adj_close_diff_^GSPC_128', 'log_volume_diff_^GSPC_128',
       'log_adj_close_diff_^GSPC_256', 'log_volume_diff_^GSPC_256',
       'log_adj_close_diff_^GSPC_512', 'log_volume_diff_^GSPC_512',
       'log_M2SL_diff_1', 'log_M2SL_diff_2', 'log_M2SL_diff_4',
       'log_M2SL_diff_8', 'log_M2SL_diff_16', 'log_M2SL_diff_32',
       'log_M2SL_diff_64', 'UNRATE_diff_1', 'UNRATE_diff_2', 'UNRATE_diff_4',
       'UNRATE_diff_8', 'UNRATE_diff_16', 'UNRATE_diff_32', 'FEDFUNDS_diff_1',
       'FEDFUNDS_diff_2', 'FEDFUNDS_diff_4', 'FEDFUNDS_

In [8]:
def get_target_name(id, future_period=256):
  return f'future_profit_{id}_{future_period}'

def get_guest_mse(df_train_X, df_train_y, id, future_period, method):
  target_name = get_target_name(id, future_period)
  if method == 'last':
    guess_mse = ((df_train_X[f'log_adj_close_diff_{id}_{future_period}'] - df_train_y[target_name])**2).sum()/len(df_train_y)
    guess_me = (df_train_y[target_name] - df_train_X[f'log_adj_close_diff_{id}_{future_period}']).sum()/len(df_train_y)
  elif method == 'zero':
    guess_mse = ((0 - df_train_y[target_name])**2).sum()/len(df_train_y)
    guess_me = (df_train_y[target_name]).sum()/len(df_train_y)
  else:
    raise Exception('error')
  return guess_mse, guess_me

In [9]:


df_train_X, df_train_y, df_test_X, df_test_y = split_shuffle(df_X, df_y)
last_mse, last_me = get_guest_mse(df_test_X, df_test_y, '^GSPC', 128, 'last')
zero_mse, zero_me = get_guest_mse(df_test_X, df_test_y, '^GSPC', 128, 'zero')

alpha = 0.8
print('MSE use data from last period:', last_mse, last_me, alpha*last_mse + (1-alpha)*last_me)
print('MSE use zero as prediction:', zero_mse, zero_me, alpha*zero_mse + (1-alpha)*zero_me)

df_train = df[:len(df_train_X)]
df_test = df[len(df_train_X):]

MSE use data from last period: 0.015174998578427214 0.010960890808180645 0.0143321770243779
MSE use zero as prediction: 0.01056648036978085 0.05921277807061188 0.020295739909947052


In [10]:
print(df_train_X.shape)
print(df_train_y.shape)

(11596, 39)
(11596, 1)


In [11]:
print(df_train_X.shape)
print(df_train_y.shape)
print(df_test_X.shape)
print(df_test_y.shape)
print(df_X.shape)
print(df.shape)

(11596, 39)
(11596, 1)
(2900, 39)
(2900, 1)
(14496, 39)
(14496, 41)


In [12]:
def eva_model(id, df_test, y_pred, lower_quartile, y_test):
  df_test['y_pred'] = y_pred
  df_test['y_test'] = y_test
  df_test['y_lower_quartile'] = lower_quartile
  df_test['1d_log_profit'] = df_test[f'log_adj_close_diff_{id}_1'].shift(-1).fillna(0)
  originProfit = np.exp(df_test['1d_log_profit'].sum())
  print('original profit:', originProfit)
  print('total days:', len(df_test))
  print('original daily avg:', originProfit/len(df_test))
  filter = df_test['y_pred']>0
  modelProfit = np.exp(df_test[filter]['1d_log_profit'].sum())
  print('model profit', modelProfit)
  print('model holding days:', len(df_test[filter]))
  print('model daily avg', modelProfit/len(df_test[filter]))
  
  filter2 = df_test['y_lower_quartile']>0
  modelProfit2 = np.exp(df_test[filter2]['1d_log_profit'].sum())
  print('model lower quartile profit', modelProfit2)
  print('model lower quartile holding days:', len(df_test[filter2]))
  print('model lower quartile daily avg', modelProfit2/len(df_test[filter2]))
  

In [13]:
len(df_test_X.head().columns)

39

In [14]:
# predict day1 mse: 0.0001 / guess 0.0002
# predict day2 mse: 0.0002 / guess 0.0004
# predict day4 mse: 0.0004 / guess 0.0009
# predict day8 mse: 0.0007 / guess 0.0017
# predict day16 mse: 0.0015 / guess 0.0033
# predict day32 mse: 0.0030 / guess 0.0064
def predict_with_std(pipeline, test_X):
    """
    Given a trained pipeline with a random forest regressor as the final step,
    returns the mean and standard deviation of the predictions.

    Parameters:
        pipeline: The trained pipeline with preprocessing steps and a random forest regressor.
        test_X: The test input features.

    Returns:
        mean_predictions: The mean of all the trees' predictions.
        std_predictions: The standard deviation of all the trees' predictions.
    """
    
    # Apply preprocessing steps on the test data
    for name, step in pipeline.steps[:-1]:
        test_X = step.transform(test_X)
        
    # Extract the random forest model from the last step of the pipeline
    model = pipeline.named_steps['regress']
    
    # Get the predictions from each tree
    all_predictions = np.array([tree.predict(test_X) for tree in model.rf.estimators_])
    
    # Calculate mean and standard deviation of predictions for each sample
    mean_predictions = np.mean(all_predictions, axis=0)
    std_predictions = np.std(all_predictions, axis=0)
    median = np.median(all_predictions, axis=0)
    lower_quartile = np.percentile(all_predictions, 25, axis=0)
    upper_quartile = np.percentile(all_predictions, 75, axis=0)
    
    return mean_predictions, std_predictions, lower_quartile, upper_quartile, median

In [15]:
import optuna
study_name = f"rf_gspc_price_volume_m2_unrate_fedfunds_cols_{df_X.shape[1]}"

#optuna.delete_study(study_name,storage="mysql://root@192.168.2.34:3306/mysql")

/home/ken/anaconda3/envs/stock/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
study = optuna.create_study(study_name=study_name,
                            storage="mysql://root@192.168.2.34:3306/mysql",
                             direction='minimize', load_if_exists=True)

[I 2024-03-06 16:30:05,524] Using an existing study with name 'rf_gspc_price_volume_m2_unrate_fedfunds_cols_39' instead of creating a new one.


In [17]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from safeRegressors import SafeRandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict


n_trails = 1
# no day of days/month/year, MSE=0.0134
# with day of week, MSE=0.014
# Only GPSC price and volume: MSE=0.009
# ONly price info: MSE=0.009
# with M2SL UNRATE AND FEDFUNDS: 10 iterations: MSE=0.007

# Assuming df_train_X, df_train_y, df_test_X, df_test_y are defined
y_train = df_train_y.copy().values.ravel()
y_test = df_test_y.copy().values.ravel()

scaler = MinMaxScaler()
X_train = pd.DataFrame(data=scaler.fit_transform(df_train_X), columns=df_train_X.columns)
X_test = pd.DataFrame(data=scaler.transform(df_test_X), columns=df_test_X.columns)

def objective(trial):
    # Define the hyperparameter configuration space
    k = trial.suggest_int('k', 5, len(X_train.columns))
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 10, 100)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 10, 50)

    # Model setup
    model = SafeRandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        bootstrap=bootstrap,
        max_leaf_nodes=max_leaf_nodes
    )

    pipeline = Pipeline([
        ('truncate', SelectKBest(f_regression, k=k)), # Adjust 'k' as needed
        ('regress', model),
    ])

    # Calculate cross-validated predictions
    predictions = cross_val_predict(pipeline, X_train, y_train, cv=5, n_jobs=5)

    # Calculate MSE
    mse = mean_squared_error(y_train, predictions)

    # Calculate ME
    me = (predictions - y_train).mean()

    # Combine MSE and ME into a single objective
    # Here, we use a weighted sum to prioritize MSE over ME
    # You can adjust the weights as needed to prioritize one metric over the other
    alpha = 0.8  # Weight for MSE (set between 0 and 1)
    beta = 1 - alpha  # Weight for ME (set between 0 and 1)
    composite_score = (alpha * mse) + (beta * abs(me))

    # mse_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=5)
    # avg_mse = -mse_scores.mean()

    # Calculate and return the objective metric (e.g., MSE)
    return composite_score

# Optuna optimization
study = optuna.create_study(study_name=f"rf_gspc_price_volume_m2_unrate_fedfunds_cols_{df_X.shape[1]}", 
                            storage="mysql://root@192.168.2.34:3306/mysql",
                             direction='minimize', load_if_exists=True)

study.optimize(objective, n_trials=n_trails) # Adjust the number of trials

# Best parameters and model
best_params = study.best_params
# rebuild the pipeline
best_pipeline = Pipeline([
        ('truncate', SelectKBest(f_regression, k=best_params['k'])), # Adjust 'k' as needed
        ('regress', SafeRandomForestRegressor(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        max_features=best_params['max_features'],
        bootstrap=best_params['bootstrap'],
        max_leaf_nodes=best_params['max_leaf_nodes']
    )),
    ])
best_pipeline.fit(X_train, y_train)


# Evaluate the best model
y_pred = best_pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# calculate the mean error
me = (y_test - y_pred).sum()/len(y_test)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Mean Error: {me}")
print(f'composition score: {alpha*mae + (1-alpha)*me}')
print(f"Best Parameters: {best_params}")


[I 2024-03-06 16:30:11,216] Using an existing study with name 'rf_gspc_price_volume_m2_unrate_fedfunds_cols_39' instead of creating a new one.


In [47]:
# Save the entire model
import pickle

from utils import save_pickle


#save_pickle(best_pipeline, 'model/ml_GSPC_rf_optuna_240305.pkl')
#save_pickle(scaler, 'model/scaler240305.pkl')

In [1]:
def objective_ridge(trial):
    # Define the hyperparameter configuration space
    k = trial.suggest_int('k', 5, len(X_train.columns))
    alpha = trial.suggest_loguniform('alpha', 1e-5, 1e3)
    max_iter = trial.suggest_int('max_iter', 100, 1000)
    solver = trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'])
    tol = trial.suggest_loguniform('tol', 1e-5, 1e-1)
    random_state = trial.suggest_int('random_state', 1, 100)

    # Model setup
    model = Ridge(
        alpha=alpha,
        max_iter=max_iter,
        solver=solver,
        tol=tol,
        random_state=random_state
    )

    pipeline = Pipeline([
        ('truncate', SelectKBest(f_regression, k=k)),
        ('regress', model),
    ])

    # Calculate cross-validated predictions
    predictions = cross_val_predict(pipeline, X_train, y_train, cv=5, n_jobs=5)

    # Calculate MSE
    mse = mean_squared_error(y_train, predictions)

    # Calculate ME
    me = (predictions - y_train).mean()

    # Combine MSE and ME into a single objective
    # Here, we use a weighted sum to prioritize MSE over ME
    # You can adjust the weights as needed to prioritize one metric over the other
    alpha = 0.8  # Weight for MSE (set between 0 and 1)
    beta = 1 - alpha  # Weight for ME (set between 0 and 1)
    composite_score = (alpha * mse) + (beta * abs(me))

    return composite_score

# Optuna optimization
n_trails=20
study_name = f"ridge_gspc_price_volume_m2_unrate_fedfunds_cols_{df_X.shape[1]}"
#optuna.delete_study(study_name,storage="mysql://root@192.168.2.34:3306/mysql")
study = optuna.create_study(study_name=study_name, 
                            storage="mysql://root@192.168.2.34:3306/mysql",
                             direction='minimize', load_if_exists=True)

study.optimize(objective_ridge, n_trials=n_trails) # Adjust the number of trials
# Best parameters and model

best_params = study.best_params
# rebuild the pipeline
best_pipeline = Pipeline([
        ('truncate', SelectKBest(f_regression, k=best_params['k'])),
        ('regress', Ridge(
        alpha=best_params['alpha'],
        max_iter=best_params['max_iter'],
        solver=best_params['solver'],
        tol=best_params['tol'],
        random_state=best_params['random_state']
    )),
    ])
best_pipeline.fit(X_train, y_train)

# Evaluate the best model
y_pred = best_pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# calculate the mean error
me = (y_test - y_pred).sum()/len(y_test)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Mean Error: {me}")
print(f'composition score: {alpha*mae + (1-alpha)*me}')
print(f"Best Parameters: {best_params}")


NameError: name 'df_X' is not defined

In [43]:
import optuna
study_name_svr = f"svr_gspc_price_volume_m2_unrate_fedfunds_cols_{df_X.shape[1]}"

#optuna.delete_study(study_name_svr,storage="mysql://root@192.168.2.34:3306/mysql")
study = optuna.create_study(study_name=study_name_svr,
                            storage="mysql://root@192.168.2.34:3306/mysql",
                             direction='minimize', load_if_exists=True)

/tmp/ipykernel_1513188/378197358.py:4: FutureWarning: delete_study(): Please give all values as keyword arguments. See https://github.com/optuna/optuna/issues/3324 for details.
  optuna.delete_study(study_name_svr,storage="mysql://root@192.168.2.34:3306/mysql")
[I 2024-03-05 18:12:55,194] A new study created in RDB with name: svr_gspc_price_volume_m2_unrate_fedfunds_cols_39


In [44]:
from sklearn.preprocessing import MinMaxScaler
from safeRegressors import SafeSVR
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
import signal

n_trials=100
scaler = MinMaxScaler()
X_train = pd.DataFrame(data=scaler.fit_transform(df_train_X), columns=df_train_X.columns)
X_test = pd.DataFrame(data=scaler.transform(df_test_X), columns=df_test_X.columns)

# Define a timeout handler
class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException()

# Set the signal handler for the SIGALRM signal
signal.signal(signal.SIGALRM, timeout_handler)

def objective(trial):
  # Define the hyperparameter configuration space
  k = trial.suggest_int('k', 2, len(X_train.columns))
  # SVR hyperparameters
  kernel = trial.suggest_categorical('kernel', ['rbf', 'linear', 'poly', 'sigmoid'])
  C = trial.suggest_float('C', 0.1, 10, log=True)
  gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

  # Model setup
  model = SafeSVR(
      kernel=kernel,
      C=C,
      gamma=gamma)

  pipeline = Pipeline([
      ('truncate', SelectKBest(f_regression, k=k)), # Adjust 'k' as needed
      ('regress', model),
  ])

  # Set a timeout for the cross-validation process
  trial_timeout = 300  # Timeout in seconds for each trial
  signal.alarm(trial_timeout)
    
  try:
    # Calculate cross-validated predictions
    predictions = cross_val_predict(pipeline, X_train, y_train, cv=5, n_jobs=5)

    # Calculate MSE
    mse = mean_squared_error(y_train, predictions)

    # Calculate ME
    me = (predictions - y_train).mean()

    # Combine MSE and ME into a single objective
    # Here, we use a weighted sum to prioritize MSE over ME
    # You can adjust the weights as needed to prioritize one metric over the other
    alpha = 0.8  # Weight for MSE (set between 0 and 1)
    beta = 1 - alpha  # Weight for ME (set between 0 and 1)
    composite_score = (alpha * mse) + (beta * abs(me))

    # mse_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=5)
    # avg_mse = -mse_scores.mean()

    # Calculate and return the objective metric (e.g., MSE)
   
  except TimeoutException:
    # If a timeout occurs, return a large MSE value
    composite_score = np.inf
  finally:
    # Cancel the alarm
    signal.alarm(0)

  # Calculate and return the objective metric (e.g., MSE)
  return composite_score

# Optuna optimization
study = optuna.create_study(study_name=study_name_svr, direction='minimize', 
                            storage="mysql://root@192.168.2.34:3306/mysql",load_if_exists=True)

study.optimize(objective, n_trials = n_trials) # Adjust the number of trials

# Best parameters and model
best_params = study.best_params
# rebuild the pipeline
best_pipeline = Pipeline([
        ('truncate', SelectKBest(f_regression, k=best_params['k'])), # Adjust 'k' as needed
        ('regress', SafeSVR(
      kernel=best_params['kernel'],
      C=best_params['C'],
      gamma=best_params['gamma']))])

best_pipeline.fit(X_train, y_train)

# Evaluate the best model
y_pred = best_pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
me = (y_test - y_pred).sum()/len(y_test)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Mean Error: {me}")
print(f"Best Parameters: {best_params}")


# Save the entire model
import pickle

from utils import save_pickle

#save_pickle(best_models['SVM'], 'model/ml_GSPC_svm.pkl')
save_pickle(best_pipeline, 'model/ml_GSPC_svr_optuna.pkl')
#save_pickle(scaler, 'model/scaler.pkl')

[I 2024-03-05 18:13:10,912] Using an existing study with name 'svr_gspc_price_volume_m2_unrate_fedfunds_cols_39' instead of creating a new one.
[I 2024-03-05 18:13:14,895] Trial 0 finished with value: 0.011241616014342478 and parameters: {'k': 11, 'kernel': 'linear', 'C': 3.655122122882126, 'gamma': 'auto'}. Best is trial 0 with value: 0.011241616014342478.
[I 2024-03-05 18:13:17,909] Trial 1 finished with value: 0.009773290309461481 and parameters: {'k': 35, 'kernel': 'rbf', 'C': 0.4209580753067123, 'gamma': 'auto'}. Best is trial 1 with value: 0.009773290309461481.
[I 2024-03-05 18:13:26,444] Trial 2 finished with value: 0.004230919164077714 and parameters: {'k': 33, 'kernel': 'poly', 'C': 0.2723156603820219, 'gamma': 'scale'}. Best is trial 2 with value: 0.004230919164077714.
[I 2024-03-05 18:13:29,818] Trial 3 finished with value: 0.01127584859546985 and parameters: {'k': 9, 'kernel': 'linear', 'C': 4.5753553875342, 'gamma': 'scale'}. Best is trial 2 with value: 0.00423091916407771

Mean Absolute Error: 0.11309076506070317
Mean Squared Error: 0.02626846336958272
Mean Error: -0.00028465950491401895
Best Parameters: {'k': 11, 'kernel': 'rbf', 'C': 9.977599861096579, 'gamma': 'scale'}


In [17]:
y_pred.max()

0.19317519413063453

In [19]:
# Save the entire model
import pickle

from utils import save_pickle

#save_pickle(best_models['SVM'], 'model/ml_GSPC_svm.pkl')
save_pickle(best_pipeline, 'model/ml_GSPC_rf_optuna.pkl')
save_pickle(scaler, 'model/scaler.pkl')

In [10]:
# Check if GPU is available and set the device accordingly
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [51]:
import optuna  
import numpy as np  
import torch  
import torch.nn as nn  
import torch.optim as optim  
from torch.utils.data import DataLoader, TensorDataset, random_split  
from sklearn.preprocessing import MinMaxScaler  
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold




In [63]:
optuna.delete_study(study_name=f"ann_gspc_price_volume_m2_unrate_fedfunds_cols_{df_X.shape[1]}_init_0.001", 
                    storage="mysql://root@192.168.2.34:3306/mysql")

In [64]:
study = optuna.create_study(study_name=f"ann_gspc_price_volume_m2_unrate_fedfunds_cols_{df_X.shape[1]}_init_0.001", 
                            storage="mysql://root@192.168.2.34:3306/mysql",
                             direction='minimize', load_if_exists=True)

[I 2024-03-06 06:56:01,441] A new study created in RDB with name: ann_gspc_price_volume_m2_unrate_fedfunds_cols_39_init_0.001


In [65]:

# Assuming df_train_X, df_train_y, df_test_X, df_test_y are already defined and loaded  
# test results:
# layer = 1-6, without day of the week/month/year, MSE=0.0133

# with all macro:  0.0133 for best model.
# no macro data 0.0074-0.011 for best models.
# only price: 0.009-0.01 for best models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
NUM_TRIAL = 100

torch.manual_seed(42)  
np.random.seed(42)  
  
scaler = MinMaxScaler()  
X_train = scaler.fit_transform(df_train_X)  
X_test = scaler.transform(df_test_X)  
y_train = df_train_y.copy().values  
y_test = df_test_y.copy().values  
  
  
# Convert data to PyTorch tensors  
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32).to(device)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32).to(device)
  
# Create a DataLoader for training, validation, and test sets  
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)  
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)  
  
# Define the DNN model  
class DNN(nn.Module):  
    def __init__(self, dim_features, hidden_units, dropout_rate):  
        super(DNN, self).__init__()  
        layers = []  
        input_dim = dim_features  
        for units in hidden_units:
            linear = nn.Linear(input_dim, units)
            
            # Initialize weights and biases to small values close to zero
            nn.init.uniform_(linear.weight, -0.001, 0.001)
            nn.init.uniform_(linear.bias, -0.001, 0.001)

            layers.append(linear)  
            layers.append(nn.Tanh())  # Using Tanh activation function 
            layers.append(nn.Dropout(dropout_rate))
            input_dim = units  

        output_layer = nn.Linear(hidden_units[-1], 1)
        
        # Initialize weights and biases of the output layer
        nn.init.uniform_(output_layer.weight, -0.001, 0.001)
        nn.init.uniform_(output_layer.bias, -0.001, 0.001)

        layers.append(output_layer)  # Output layer

        self.model = nn.Sequential(*layers)  
  
    def forward(self, x):  
        return self.model(x)  

def composite_loss(outputs, targets, alpha=0.8):
    mse = nn.MSELoss()(outputs, targets)
    me = torch.mean(outputs - targets)
    return alpha * mse + (1-alpha) * abs(me)

def objective(trial):
    # Hyperparameters to be optimized by Optuna
    lr = trial.suggest_categorical("lr", [1e-4, 0.5e-3, 1e-3])
    hidden_layers = trial.suggest_int("hidden_layers", 1, 4)
    hidden_units = []
    for i in range(hidden_layers):
        # Suggest the number of units for the i-th layer
        units = trial.suggest_categorical(f"hidden_units_l{i}", [4, 8, 16, 32, 64, 128])
        hidden_units.append(units)
        
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256, 512, 1024])
    num_epochs = trial.suggest_categorical("num_epochs", [50, 100, 150, 200])
    dropout_rate = trial.suggest_categorical("dropout_rate", [i/100 for i in range(21)])
    
    # K-Fold Cross-Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    val_losses = []

    for train_index, val_index in kf.split(X_train):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        # Convert data to PyTorch tensors for this fold
        X_train_tensor_fold = torch.tensor(X_train_fold, dtype=torch.float32).to(device)
        y_train_tensor_fold = torch.tensor(y_train_fold.reshape(-1, 1), dtype=torch.float32).to(device)
        X_val_tensor_fold = torch.tensor(X_val_fold, dtype=torch.float32).to(device)
        y_val_tensor_fold = torch.tensor(y_val_fold.reshape(-1, 1), dtype=torch.float32).to(device)

        # DataLoader for the current fold
        train_loader_fold = DataLoader(TensorDataset(X_train_tensor_fold, y_train_tensor_fold), batch_size=batch_size, shuffle=True, num_workers=0)
        val_loader_fold = DataLoader(TensorDataset(X_val_tensor_fold, y_val_tensor_fold), batch_size=batch_size, num_workers=0)

        model = DNN(X_train.shape[1], hidden_units, dropout_rate).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Training loop for this fold
        for epoch in range(num_epochs):
            model.train()
            for inputs, targets in train_loader_fold: 
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = composite_loss(outputs, targets)
                loss.backward()
                optimizer.step()

        # Validation phase for this fold
        model.eval()
        val_loss_fold = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader_fold:
                outputs = model(inputs)
                val_loss_fold += composite_loss(outputs, targets).item()
        val_losses.append(val_loss_fold / len(val_loader_fold))

    # Average validation loss across all folds
    avg_val_loss = np.mean(val_losses)
    return avg_val_loss
  


cuda


In [66]:
# Optuna optimization
study = optuna.create_study(study_name=f"ann_gspc_price_volume_m2_unrate_fedfunds_cols_{df_X.shape[1]}_init_0.001",
                            storage="mysql://root@192.168.2.34:3306/mysql",
                             direction='minimize', load_if_exists=True)

study.optimize(objective, n_trials=NUM_TRIAL)  

print("Number of finished trials: ", len(study.trials))  
print("Best trial:")  
trial = study.best_trial  
  
print("  Value: ", trial.value)  
print("  Params: ")  
for key, value in trial.params.items():  
    print(f"    {key}: {value}")  


[I 2024-03-06 06:56:04,378] Using an existing study with name 'ann_gspc_price_volume_m2_unrate_fedfunds_cols_39_init_0.001' instead of creating a new one.
[I 2024-03-06 07:00:25,845] Trial 0 finished with value: 0.012829862791742827 and parameters: {'lr': 0.0005, 'hidden_layers': 2, 'hidden_units_l0': 32, 'hidden_units_l1': 128, 'batch_size': 64, 'num_epochs': 100, 'dropout_rate': 0.08}. Best is trial 0 with value: 0.012829862791742827.
[I 2024-03-06 07:02:34,420] Trial 1 finished with value: 0.012047569006681441 and parameters: {'lr': 0.0005, 'hidden_layers': 4, 'hidden_units_l0': 32, 'hidden_units_l1': 4, 'hidden_units_l2': 4, 'hidden_units_l3': 16, 'batch_size': 512, 'num_epochs': 150, 'dropout_rate': 0.17}. Best is trial 1 with value: 0.012047569006681441.
[I 2024-03-06 07:04:48,779] Trial 2 finished with value: 0.013350643914797016 and parameters: {'lr': 0.0001, 'hidden_layers': 2, 'hidden_units_l0': 8, 'hidden_units_l1': 8, 'batch_size': 64, 'num_epochs': 50, 'dropout_rate': 0.16

Number of finished trials:  100
Best trial:
  Value:  0.011560645575324694
  Params: 
    lr: 0.001
    hidden_layers: 1
    hidden_units_l0: 8
    batch_size: 1024
    num_epochs: 150
    dropout_rate: 0.0


In [67]:
study.trials

[FrozenTrial(number=0, state=1, values=[0.012829862791742827], datetime_start=datetime.datetime(2024, 3, 6, 6, 56, 4), datetime_complete=datetime.datetime(2024, 3, 6, 7, 0, 26), params={'lr': 0.0005, 'hidden_layers': 2, 'hidden_units_l0': 32, 'hidden_units_l1': 128, 'batch_size': 64, 'num_epochs': 100, 'dropout_rate': 0.08}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': CategoricalDistribution(choices=(0.0001, 0.0005, 0.001)), 'hidden_layers': IntDistribution(high=4, log=False, low=1, step=1), 'hidden_units_l0': CategoricalDistribution(choices=(4, 8, 16, 32, 64, 128)), 'hidden_units_l1': CategoricalDistribution(choices=(4, 8, 16, 32, 64, 128)), 'batch_size': CategoricalDistribution(choices=(64, 128, 256, 512, 1024)), 'num_epochs': CategoricalDistribution(choices=(50, 100, 150, 200)), 'dropout_rate': CategoricalDistribution(choices=(0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2))},

In [69]:
# Optuna optimization

import optuna
from utils import save_pickle
study = optuna.create_study(study_name=f"ann_gspc_price_volume_m2_unrate_fedfunds_cols_{df_X.shape[1]}_init_0.001", 
                            storage="mysql://root@192.168.2.34:3306/mysql",
                             direction='minimize', load_if_exists=True)

sorted_trials = sorted(study.trials, key=lambda trial: trial.value if trial.value is not None else float('inf'))

top_trials = sorted_trials[:10] 
criterion = nn.MSELoss()
for i, trial in enumerate(top_trials):
  print(f'Training the No. {i+1} model...')
  print("  Value: ", trial.value)  
  print("  Params: ")
  for key, value in trial.params.items():
      print(f"    {key}: {value}")
  
  hidden_units = [trial.params[f"hidden_units_l{i}"] for i in range(trial.params["hidden_layers"])]
  model = DNN(X_train.shape[1], hidden_units, trial.params["dropout_rate"]).to(device)
  optimizer = optim.Adam(model.parameters(), lr=trial.params["lr"])
  num_epochs = trial.params["num_epochs"]
  
  # Retrain the model with the best hyperparameters on the full training set (train + validation)  
  full_train_loader = DataLoader(TensorDataset(
      torch.tensor(X_train, dtype=torch.float32).to(device),
      torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32).to(device)
  ), batch_size=trial.params["batch_size"], shuffle=True)

  # Training loop  
  for epoch in range(num_epochs):  
    model.train()  
    for inputs, targets in full_train_loader:  
      inputs, targets = inputs.to(device), targets.to(device)
      optimizer.zero_grad()  
      outputs = model(inputs)  
      loss = composite_loss(outputs, targets)
      loss.backward()  
      optimizer.step()
    
  # Final evaluation on the test set  
  test_loader = DataLoader(test_dataset, batch_size=trial.params["batch_size"])  
  model.eval()  
  test_loss = 0.0
  with torch.no_grad():
    for inputs, targets in test_loader:
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = model(inputs)  
      test_loss += composite_loss(outputs, targets)
    
  mse_test = test_loss / len(test_loader)  
  print("Test MSE:", mse_test)  
  
  # save the model
  save_pickle(model, f'model/GSPC_ANN_optuna_{i}.pkl')

[I 2024-03-06 12:54:41,679] Using an existing study with name 'ann_gspc_price_volume_m2_unrate_fedfunds_cols_39_init_0.001' instead of creating a new one.


Training the No. 1 model...
  Value:  0.011560645575324694
  Params: 
    lr: 0.001
    hidden_layers: 1
    hidden_units_l0: 8
    batch_size: 1024
    num_epochs: 150
    dropout_rate: 0.0
Test MSE: tensor(0.0114, device='cuda:0')
Training the No. 2 model...
  Value:  0.01156253187606732
  Params: 
    lr: 0.001
    hidden_layers: 1
    hidden_units_l0: 8
    batch_size: 1024
    num_epochs: 150
    dropout_rate: 0.09
Test MSE: tensor(0.0121, device='cuda:0')
Training the No. 3 model...
  Value:  0.011577176240583261
  Params: 
    lr: 0.001
    hidden_layers: 1
    hidden_units_l0: 8
    batch_size: 1024
    num_epochs: 150
    dropout_rate: 0.05
Test MSE: tensor(0.0125, device='cuda:0')
Training the No. 4 model...
  Value:  0.011599500042696794
  Params: 
    lr: 0.001
    hidden_layers: 1
    hidden_units_l0: 8
    batch_size: 1024
    num_epochs: 150
    dropout_rate: 0.09
Test MSE: tensor(0.0118, device='cuda:0')
Training the No. 5 model...
  Value:  0.011602515913546085
  Param

In [34]:
# Save the entire model
import pickle
  
def save_pickle(obj, filename):
  # Save the scaler object to a file
  with open(filename, 'wb') as file:
      pickle.dump(obj, file)

#save_pickle(best_models['SVM'], 'model/ml_GSPC_svm.pkl')
save_pickle(model, 'model/GSPC_ANN_optuna.pkl')
save_pickle(scaler, 'model/scaler.pkl')

In [35]:
best_trials = sorted(study.trials, key=lambda trial: trial.value)[:10] 
best_trials 

In [35]:


torch.save(model, 'model/GSPC_.pth')
save_pickle(full_pipeline, 'model/ml_GSPC,pkl')
save_pickle(scaler, 'model/scaler.pkl')


In [88]:
df_test = pd.concat([df_test_X, df_test_y], axis=1)
df_test['GSPC_pred'] = y_pred

In [89]:


id = 'MSFT'
df_stock1, column_names = get_table_by_id_yahoo(id, 10, start='2017-01-01')

df_stock = df_stock1.merge(df_test['GSPC_pred'], left_index=True, right_index=True)
df_stock.loc[df_stock['GSPC_pred']<0, 'GSPC_pred']=0

df_stock['1d_log_profit'] = df_stock[f'log_adj_close_diff_{id}_1'].shift(-1).fillna(0)
df_stock['1d_profit'] = np.exp(df_stock['1d_log_profit'])-1
print('original profit:', np.exp(df_stock['1d_log_profit'].sum()))
print('total days', len(df_stock))

filter = df_stock['GSPC_pred']>0
modelProfit = np.exp(df_stock[filter]['1d_log_profit'].sum())
print('model profit', modelProfit)
print('model holding days:', len(df_stock[filter]))
print('model daily avg', modelProfit/len(df_stock[filter]))



[*********************100%***********************]  1 of 1 completed
original profit: 4.1319891622285905
total days 1505
model profit 5.0923741342105036
model holding days: 872
model daily avg 0.0058398785942781


In [17]:
df_stock['weight'].sum()

1377.0

[*********************100%***********************]  1 of 1 completed


KeyError: 'GSPC_pred'

In [32]:
hidden_node_names = []
for i in range(hiddens.shape[1]):
  name = f'hidden_nodes_{i}'
  df[name] = hiddens[:,i]
  hidden_node_names.append(name)
hidden_node_names

AttributeError: 'list' object has no attribute 'shape'

In [1128]:
test_stock_neural('MSFT', df, hidden_node_names, n_epochs=600)

[*********************100%***********************]  1 of 1 completed
Nan rows from top: 512
Nan rows from bottom:  257
guess_mae is 0.33703659729778507
start to train the model
(1725, 28)
Epoch 20/600 - Loss: 0.2880
Epoch 40/600 - Loss: 0.2846
Epoch 60/600 - Loss: 0.2841
Epoch 80/600 - Loss: 0.2838
Epoch 100/600 - Loss: 0.2837
Epoch 120/600 - Loss: 0.2827
Epoch 140/600 - Loss: 0.2824
Epoch 160/600 - Loss: 0.2829
Epoch 180/600 - Loss: 0.2826
Epoch 200/600 - Loss: 0.2823
Epoch 220/600 - Loss: 0.2824
Epoch 240/600 - Loss: 0.2818
Epoch 260/600 - Loss: 0.2818
Epoch 280/600 - Loss: 0.2819
Epoch 300/600 - Loss: 0.2827
Epoch 320/600 - Loss: 0.2819
Epoch 340/600 - Loss: 0.2813
Epoch 360/600 - Loss: 0.2822
Epoch 380/600 - Loss: 0.2810
Epoch 400/600 - Loss: 0.2805
Epoch 420/600 - Loss: 0.2809
Epoch 440/600 - Loss: 0.2811
Epoch 460/600 - Loss: 0.2811
Epoch 480/600 - Loss: 0.2808
Epoch 500/600 - Loss: 0.2798
Epoch 520/600 - Loss: 0.2801
Epoch 540/600 - Loss: 0.2796
Epoch 560/600 - Loss: 0.2797
Epoc

In [1119]:
def test_stock_ml(id, df_base_X, hidden_node_names, start='1980-01-01', n_epochs=50):
  # Now load stock.
  df_stock, column_names = get_table_by_id_yahoo(id, 10, start=start)
  df_stock, target_name = add_target(df_stock, id)
  df_stock = df_stock.merge(df_base_X[hidden_node_names], how='left', left_index=True, right_index=True)
  df_stock_X = df_stock[column_names + hidden_node_names]
  #df_stock_X = df_stock[column_names]
  df_stock_y = df_stock[[target_name]]
  df_stock, df_stock_X, df_stock_y = remove_nan(df_stock, df_stock_X, df_stock_y)
  df_stock_train_X, df_stock_train_y, df_stock_test_X, df_stock_test_y, guess_mse = split_shuffle(df_stock_X, df_stock_y, id)
  print('guess_mse is', guess_mse)
  
  print('start to train the model')
  y_train = df_stock_train_y.copy()
  y_test = df_stock_test_y.copy()
  # Scale the features
  scaler = StandardScaler()
  X_train = scaler.fit_transform(df_stock_train_X)
  X_test = scaler.transform(df_stock_test_X)

  # Train the SVM model
  #svm = SVR(kernel='rbf')  # 0.18
  #svm = SVR(kernel='rbf', C=0.1)  #0.028697 only S&P 500 history  0.0265 added M2 #0.0226 add unrate

  #ml_model = SVR(kernel='rbf', C=0.01)
  #ml_model = SVR(kernel='linear', C=0.1)
  ml_model = RandomForestRegressor(n_estimators=100, max_depth=50)
  #ml_model = xg.XGBRegressor(objective ='reg:squarederror', n_estimators = 100, seed = 123)

  #svm = SVR(kernel='rbf', C=10)  # looser regulation 0.24
  #svm = SVR(kernel='linear', C=0.1) 

  ml_model.fit(X_train, y_train)
  # Make predictions
  y_pred = ml_model.predict(X_test)

  # Evaluate the model
  mse = mean_absolute_error(y_test, y_pred)
  print(f"Mean Squared Error: {mse}")
  
  eva_model(id, df_stock, y_pred, y_test)

In [1120]:
test_stock_ml('MSFT', df, hidden_node_names, n_epochs=500)

[*********************100%***********************]  1 of 1 completed
Nan rows from top: 512
Nan rows from bottom:  257
guess_mse is 0.3370365912062879
start to train the model


/tmp/ipykernel_686440/3798216473.py:33: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Mean Squared Error: 0.32984741185244415
original profit: 8.293647527033118
weighted profit 4.544959174402671
total days 1725
holding days 1708


In [1075]:
import pandas as pd
pd.options.plotting.backend = "plotly"
df_test = df_test_y.copy()
df_test['y_pred'] = y_pred
df_test['y_test'] = y_test

df_test[['y_pred', 'y_test']].plot()


In [1034]:
# Make predictions
y_pred = ml_model.predict(np.vstack([X_train, X_test]))
y_pred.shape

/home/ken/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but SVR was fitted with feature names



(14375,)

In [1035]:
df['y_pred']=y_pred


In [1046]:
test_stock('MSFT', df)

[*********************100%***********************]  1 of 1 completed
Nan rows from top: 512
Nan rows from bottom:  257
guess_mse is 0.21362961331161565
start to train the model


/home/ken/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



Mean Squared Error: 0.06203668539850105
original profit: 8.29364683811505
weighted profit 4.195818427687407
total days 1725
holding days 1725


(4313, 43)
Epoch 1/50 - Loss: 0.0303
Epoch 2/50 - Loss: 0.0270
Epoch 3/50 - Loss: 0.0259
Epoch 4/50 - Loss: 0.0250
Epoch 5/50 - Loss: 0.0243
Epoch 6/50 - Loss: 0.0238
Epoch 7/50 - Loss: 0.0236
Epoch 8/50 - Loss: 0.0234
Epoch 9/50 - Loss: 0.0232
Epoch 10/50 - Loss: 0.0232
Epoch 11/50 - Loss: 0.0231
Epoch 12/50 - Loss: 0.0229
Epoch 13/50 - Loss: 0.0229
Epoch 14/50 - Loss: 0.0227
Epoch 15/50 - Loss: 0.0227
Epoch 16/50 - Loss: 0.0227
Epoch 17/50 - Loss: 0.0226
Epoch 18/50 - Loss: 0.0225
Epoch 19/50 - Loss: 0.0224
Epoch 20/50 - Loss: 0.0224
Epoch 21/50 - Loss: 0.0222
Epoch 22/50 - Loss: 0.0221
Epoch 23/50 - Loss: 0.0221
Epoch 24/50 - Loss: 0.0221
Epoch 25/50 - Loss: 0.0218
Epoch 26/50 - Loss: 0.0218
Epoch 27/50 - Loss: 0.0217
Epoch 28/50 - Loss: 0.0214
Epoch 29/50 - Loss: 0.0214
Epoch 30/50 - Loss: 0.0213
Epoch 31/50 - Loss: 0.0211
Epoch 32/50 - Loss: 0.0210
Epoch 33/50 - Loss: 0.0210
Epoch 34/50 - Loss: 0.0209
Epoch 35/50 - Loss: 0.0209
Epoch 36/50 - Loss: 0.0206
Epoch 37/50 - Loss: 0.0205

In [883]:
df_test = df[len(df_train_X):].copy()
df_test['y_pred'] = y_pred
df_test['y_test'] = y_test

df_test['1y_profit'] = np.exp(df_test['y_pred'])-1

df_test.loc[df_test['1y_profit']<0, '1y_profit']=0 # type: ignore
df_test['weight'] = df_test['1y_profit']*len(df_test)/df_test['1y_profit'].sum()


df_test['1d_log_profit'] = df_test['log_adj_close_diff_^GSPC_1'].shift(-1).fillna(0)
df_test['1d_profit'] = np.exp(df_test['1d_log_profit'])-1
df_test['weighted_1d_profit'] = df_test['1d_profit'] * df_test['weight']
df_test['log_weighted_1d_profit'] = np.log(1+df_test['weighted_1d_profit'])
print(np.exp(df_test['1d_profit'].sum()))
print(np.exp(df_test['log_weighted_1d_profit'].sum()))

print(df_test['weight'].sum())


4.983824537777031
2.164725896387563
4313.0


In [804]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'log_adj_close_^GSPC', 'last_log_adj_close_^GSPC', 'log_volume_^GSPC',
       'log_adj_close_diff_^GSPC_1', 'log_volume_diff_^GSPC_1',
       'log_adj_close_diff_^GSPC_2', 'log_volume_diff_^GSPC_2',
       'log_adj_close_diff_^GSPC_4', 'log_volume_diff_^GSPC_4',
       'log_adj_close_diff_^GSPC_8', 'log_volume_diff_^GSPC_8',
       'log_adj_close_diff_^GSPC_16', 'log_volume_diff_^GSPC_16',
       'log_adj_close_diff_^GSPC_32', 'log_volume_diff_^GSPC_32',
       'log_adj_close_diff_^GSPC_64', 'log_volume_diff_^GSPC_64',
       'log_adj_close_diff_^GSPC_128', 'log_volume_diff_^GSPC_128',
       'log_adj_close_diff_^GSPC_256', 'log_volume_diff_^GSPC_256',
       'log_adj_close_diff_^GSPC_512', 'log_volume_diff_^GSPC_512',
       'future_profit_^GSPC_256', 'hidden_nodes_0', 'hidden_nodes_1'],
      dtype='object')

4.983824537777031
2.0847471057004787
4313.0


In [841]:
df_test['weight']

Date
2005-04-22 00:00:00-04:00    1.163827
2005-04-25 00:00:00-04:00    1.148805
2005-04-26 00:00:00-04:00    1.176177
2005-04-27 00:00:00-04:00    1.200482
2005-04-28 00:00:00-04:00    1.215179
                               ...   
2022-06-02 00:00:00-04:00    0.282926
2022-06-03 00:00:00-04:00    0.184279
2022-06-06 00:00:00-04:00    0.197415
2022-06-07 00:00:00-04:00    0.229714
2022-06-08 00:00:00-04:00    0.211428
Name: weight, Length: 4313, dtype: float32

In [ ]:

df_test['weight'] = np.exp(df_test['y_pred'])-1
df_test.loc[df_test['weight']<0, 'weight'] = 0 # type: ignore
weight_sum = df_test['weight'].sum()
df_test['weight'] = df_test['weight']*len(df_test)/weight_sum
#df_test['y_pred_adj'] = y_pred * (len(df_test)/y_pred.sum()) # type: ignore

df_test['1d_profit'] = df_test['log_adj_close_diff_^GSPC_1'].shift(-1).fillna(0)
#df_test['weighted_profit'] = np.log((np.exp(df_test['1d_profit'])-1)*df_test['weight']+1)
df_test['weighted_profit'] = df_test['1d_profit']*df_test['weight']
print(np.exp(df_test['1d_profit'].sum()))
print(np.exp(df_test['weighted_profit'].sum()))
print(len(df_test))

In [837]:
df_test['weight'].sum()

4313.0

In [829]:
df_test['1d_profit']

Date
2005-04-22 00:00:00-04:00    0.008625
2005-04-25 00:00:00-04:00   -0.008877
2005-04-26 00:00:00-04:00    0.003942
2005-04-27 00:00:00-04:00   -0.011446
2005-04-28 00:00:00-04:00    0.011852
                               ...   
2022-06-02 00:00:00-04:00   -0.016482
2022-06-03 00:00:00-04:00    0.003132
2022-06-06 00:00:00-04:00    0.009478
2022-06-07 00:00:00-04:00   -0.010853
2022-06-08 00:00:00-04:00    0.000000
Name: 1d_profit, Length: 4313, dtype: float64

In [835]:
df_test['weight']

Date
2005-04-22 00:00:00-04:00    1.163827
2005-04-25 00:00:00-04:00    1.148805
2005-04-26 00:00:00-04:00    1.176177
2005-04-27 00:00:00-04:00    1.200482
2005-04-28 00:00:00-04:00    1.215179
                               ...   
2022-06-02 00:00:00-04:00    0.282926
2022-06-03 00:00:00-04:00    0.184279
2022-06-06 00:00:00-04:00    0.197415
2022-06-07 00:00:00-04:00    0.229714
2022-06-08 00:00:00-04:00    0.211428
Name: weight, Length: 4313, dtype: float32

In [825]:
np.exp(df_test['y_pred'])-1

Date
2005-04-22 00:00:00-04:00    0.097058
2005-04-25 00:00:00-04:00    0.095806
2005-04-26 00:00:00-04:00    0.098088
2005-04-27 00:00:00-04:00    0.100115
2005-04-28 00:00:00-04:00    0.101341
                               ...   
2022-06-02 00:00:00-04:00    0.023595
2022-06-03 00:00:00-04:00    0.015368
2022-06-06 00:00:00-04:00    0.016464
2022-06-07 00:00:00-04:00    0.019157
2022-06-08 00:00:00-04:00    0.017632
Name: y_pred, Length: 4313, dtype: float32

In [819]:
(np.exp(df_test['1d_profit'])-1) *  df_test['weight']

Date
2005-04-22 00:00:00-04:00    0.010284
2005-04-25 00:00:00-04:00   -0.010362
2005-04-26 00:00:00-04:00    0.004737
2005-04-27 00:00:00-04:00   -0.013916
2005-04-28 00:00:00-04:00    0.014749
                               ...   
2022-06-02 00:00:00-04:00   -0.004886
2022-06-03 00:00:00-04:00    0.000613
2022-06-06 00:00:00-04:00    0.001993
2022-06-07 00:00:00-04:00   -0.002625
2022-06-08 00:00:00-04:00    0.000000
Length: 4313, dtype: float64

In [821]:
np.exp(df_test['1d_profit'])

Date
2005-04-22 00:00:00-04:00    1.008662
2005-04-25 00:00:00-04:00    0.991163
2005-04-26 00:00:00-04:00    1.003950
2005-04-27 00:00:00-04:00    0.988620
2005-04-28 00:00:00-04:00    1.011922
                               ...   
2022-06-02 00:00:00-04:00    0.983653
2022-06-03 00:00:00-04:00    1.003137
2022-06-06 00:00:00-04:00    1.009523
2022-06-07 00:00:00-04:00    0.989206
2022-06-08 00:00:00-04:00    1.000000
Name: 1d_profit, Length: 4313, dtype: float64

In [809]:
df_test[['y_pred','weight']]

,y_pred,weight
Date,,
2005-04-22 00:00:00-04:00,0.092632,1.187184
2005-04-25 00:00:00-04:00,0.091490,1.172540
2005-04-26 00:00:00-04:00,0.093571,1.199211
2005-04-27 00:00:00-04:00,0.095415,1.222846
2005-04-28 00:00:00-04:00,0.096528,1.237116
...,...,...
2022-06-02 00:00:00-04:00,0.023321,0.298881
2022-06-03 00:00:00-04:00,0.015251,0.195460
2022-06-06 00:00:00-04:00,0.016330,0.209281


In [810]:
df_test['weight'].sum()

4313.0005

In [771]:
df_test.head()

,Open,High,Low,Close,Adj Close,Volume,log_adj_close_^GSPC,last_log_adj_close_^GSPC,log_volume_^GSPC,log_adj_close_diff_^GSPC_1,...,log_adj_close_diff_^GSPC_512,log_volume_diff_^GSPC_512,future_profit_^GSPC_256,hidden_nodes_0,hidden_nodes_1,y_pred,y_test,weight,1d_profit,weighted_profit
Date,,,,,,,,,,,,,,,,,,,,,
2005-04-22 00:00:00-04:00,1159.949951,1159.949951,1142.949951,1152.119995,1152.119995,2045880000,7.049359,7.056132,21.439094,-0.006773,...,0.279051,0.472647,0.128889,0.281322,0.071097,0.083673,0.128889,1.057550,0.008625,0.009121
2005-04-25 00:00:00-04:00,1152.119995,1164.050049,1152.119995,1162.099976,1162.099976,1795030000,7.057984,7.049359,21.308288,0.008625,...,0.291447,0.452591,0.116120,0.290203,0.076558,0.082052,0.116120,1.055837,-0.008877,-0.009372
2005-04-26 00:00:00-04:00,1162.099976,1164.800049,1151.829956,1151.829956,1151.829956,1959740000,7.049107,7.057984,21.396078,-0.008877,...,0.263260,0.549710,0.131123,0.285194,0.078958,0.084155,0.131123,1.058060,0.003942,0.004171
2005-04-27 00:00:00-04:00,1151.739990,1159.869995,1144.420044,1156.380005,1156.380005,2151520000,7.053050,7.049107,21.489440,0.003942,...,0.260919,0.387601,0.123297,0.289914,0.090476,0.085163,0.123297,1.059127,-0.011446,-0.012122
2005-04-28 00:00:00-04:00,1156.380005,1156.380005,1143.219971,1143.219971,1143.219971,2182270000,7.041604,7.053050,21.503631,-0.011446,...,0.261784,0.318142,0.137894,0.274339,0.077718,0.087329,0.137894,1.061423,0.011852,0.012580


In [744]:
# calculate the hidden unit.
y_pred = []
hiddens = []
test_loss = 0
with torch.no_grad():
  for inputs, targets in train_loader:
    outputs, hidden = model(inputs)
    y_pred.append(outputs.detach().numpy())
    hiddens.append(hidden.detach().numpy())
    test_loss += criterion(outputs, targets).item()
  mse = test_loss / len(train_loader) 
  print(f"Mean Squared train Error: {mse}")
  
  test_loss = 0
  for inputs, targets in test_loader:
    outputs, hidden = model(inputs)
    y_pred.append(outputs.detach().numpy())
    hiddens.append(hidden.detach().numpy())
    test_loss += criterion(outputs, targets).item()
  mse = test_loss / len(test_loader) 
  print(f"Mean Squared test Error: {mse}")

y_pred = np.concatenate(y_pred)
hiddens = np.concatenate(hiddens)
print(hiddens.shape)

Mean Squared train Error: 0.023886038549244405
Mean Squared test Error: 0.028757247142493725
(14375, 2)


In [746]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_test = df[len(df_train_X):].copy()
df_test['y_pred'] = y_pred[len(df_train_X):]
df_test['y_test'] = y_test


pd.options.plotting.backend = "plotly"
# Create subplots
fig = make_subplots(rows=2, cols=1, row_heights=[0.6, 0.4])
# Add traces to the subplots
fig.add_trace(go.Scatter(x=df_test.index, y=df_test['y_pred'], name='y_pred'), row=1, col=1)
fig.add_trace(go.Scatter(x=df_test.index, y=df_test['y_test'], name='y_test'), row=1, col=1)
fig.add_trace(go.Scatter(x=df_test.index, y=df_test['log_adj_close_^GSPC'], name='adj_close'), row=2, col=1)

# Update layout and display the plot
fig.update_layout(height=600, width=800, title_text="Two Subplots")
fig.show()



#df_test[['y_pred', 'y_test']].plot()

In [758]:
def test_stock(id, df_base, start='1980-01-01', n_epochs=50):
  # Now load stock.
  df_stock, column_names = get_table_by_id_yahoo(id, 10, start=start)
  df_stock, target_name = add_target(df_stock, id)
  df_stock = df_stock.merge(df[hidden_node_names], how='left', left_index=True, right_index=True)
  df_stock_X = df_stock[column_names + hidden_node_names]
  #df_stock_X = df_stock[column_names]
  df_stock_y = df_stock[[target_name]]
  df_stock, df_stock_X, df_stock_y = remove_nan(df_stock, df_stock_X, df_stock_y)
  df_stock_train_X, df_stock_train_y, df_stock_test_X, df_stock_test_y, guess_mse = split_shuffle(df_stock_X, df_stock_y, id)
  print('guess_mse is', guess_mse)
  
  print('start to train the model')
  
  torch.manual_seed(42)
  np.random.seed(42)
  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(df_stock_train_X)
  X_test = scaler.transform(df_stock_test_X)
  y_train = df_stock_train_y.copy().values

  print(X_test.shape)

  y_test = df_stock_test_y.copy().values
  # Convert data to PyTorch tensors
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
  y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
  y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)

  # Create a DataLoader for training and test sets
  train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
  test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
  batch_size = 512
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size)

  # Define the DNN model
  class DNN(nn.Module):
      def __init__(self, dim_features, n_hidden_nodes):
          super(DNN, self).__init__()
          self.fc1 = nn.Linear(dim_features, n_hidden_nodes)
          self.fc2 = nn.Linear(n_hidden_nodes, 1)


      def forward(self, x):
          #x = torch.relu(self.fc1(x))
          x1 = self.fc1(x)
          x1_tanh = torch.tanh(x1)
          x2 = self.fc2(x1_tanh)
          return x2, x1

  # Create an instance of the DNN model
  dim_features = X_train.shape[1]
  model = DNN(dim_features, 2)

  # Define the loss function and optimizer
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.0001)

  # Train the model
  num_epochs = 50
  for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in train_loader:
      optimizer.zero_grad()
      outputs, hidden = model(inputs)
      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
        
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}")

  # Evaluate the model
  model.eval()
  test_loss = 0.0
  y_pred = []
  hiddens = []
  with torch.no_grad():
    for inputs, targets in test_loader:
      outputs, hidden = model(inputs)
      y_pred.append(outputs.detach().numpy())
      hiddens.append(hidden.detach().numpy())
      test_loss += criterion(outputs, targets).item()

  print(len(test_loader))
  mse = test_loss / len(test_loader)
  print(f"Mean Squared Error for hidden_nodes: {mse}")
  y_pred = np.concatenate(y_pred)
  y_pred_bast = y_pred
  df_stock_test = df_stock[len(df_stock_train_X):].copy()
  df_stock_test['y_pred'] = y_pred
  df_stock_test['y_test'] = y_test
  print('y_pred_max', y_pred.max())
  df_stock_test['weight'] = np.exp(df_stock_test['y_pred'])
  df_stock_test.loc[df_test['y_pred']<0, 'weight'] = 0 # type: ignore
  weight_sum = df_stock_test['weight'].sum()
  df_stock_test['weight'] = df_stock_test['weight']*len(df_stock_test)/weight_sum
  
  #df_stock_test.loc[df_stock_test['y_pred']<0, 'y_pred'] = 0 # type: ignore
  #df_stock_test['y_pred_adj'] = y_pred * (len(df_stock_test)/y_pred.sum()) # type: ignore

  df_stock_test['1d_profit'] = df_stock_test[f'log_adj_close_diff_{id}_1'].shift(-1).fillna(0)
  df_stock_test['weighted_profit'] = df_stock_test['1d_profit'] * df_stock_test['weight']
  print(np.exp(df_stock_test['1d_profit'].sum()))
  print(np.exp(df_stock_test['weighted_profit'].sum()))
  print(df_stock_test['weight'].sum())
  print(len(df_stock_test))
  

In [765]:
test_stock('GOOGL', df)

[*********************100%***********************]  1 of 1 completed
Nan rows from top: 512
Nan rows from bottom:  257
guess_mse is 0.16693518984521521
start to train the model
(1192, 22)
Epoch 1/50 - Loss: 0.2785
Epoch 2/50 - Loss: 0.2730
Epoch 3/50 - Loss: 0.2723
Epoch 4/50 - Loss: 0.2685
Epoch 5/50 - Loss: 0.2636
Epoch 6/50 - Loss: 0.2618
Epoch 7/50 - Loss: 0.2541
Epoch 8/50 - Loss: 0.2534
Epoch 9/50 - Loss: 0.2484
Epoch 10/50 - Loss: 0.2478
Epoch 11/50 - Loss: 0.2400
Epoch 12/50 - Loss: 0.2389
Epoch 13/50 - Loss: 0.2363
Epoch 14/50 - Loss: 0.2331
Epoch 15/50 - Loss: 0.2284
Epoch 16/50 - Loss: 0.2243
Epoch 17/50 - Loss: 0.2230
Epoch 18/50 - Loss: 0.2204
Epoch 19/50 - Loss: 0.2165
Epoch 20/50 - Loss: 0.2146
Epoch 21/50 - Loss: 0.2079
Epoch 22/50 - Loss: 0.2073
Epoch 23/50 - Loss: 0.2061
Epoch 24/50 - Loss: 0.2045
Epoch 25/50 - Loss: 0.1985
Epoch 26/50 - Loss: 0.1949
Epoch 27/50 - Loss: 0.1924
Epoch 28/50 - Loss: 0.1868
Epoch 29/50 - Loss: 0.1873
Epoch 30/50 - Loss: 0.1829
Epoch 31/50

In [664]:
# Now load stock.
df_stock, column_names = get_table_by_id_yahoo('AAPL', 10, start='1981-01-01')
df_stock, target_name = add_target(df_stock, 'AAPL')
#df_stock = df_appl.merge(df[hidden_node_names], how='left', left_index=True, right_index=True)
#df_appl_X = df_appl[column_names + hidden_node_names]
df_appl_X = df_appl[column_names]
df_appl_y = df_appl[[target_name]]
df_appl, df_appl_X, df_appl_y = remove_nan(df_appl, df_appl_X, df_appl_y)
df_appl_train_X, df_appl_train_y, df_appl_test_X, df_appl_test_y, guess_mse = split_shuffle(df_appl_X, df_appl_y, 'AAPL')
print(guess_mse)

[*********************100%***********************]  1 of 1 completed
Nan rows from top: 512
Nan rows from bottom:  257
0.5871811523570127


In [665]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

torch.manual_seed(42)
np.random.seed(42)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(df_appl_train_X)
X_test = scaler.transform(df_appl_test_X)
y_train = df_appl_train_y.copy().values

print(X_test.shape)

y_test = df_appl_test_y.copy().values
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)

# Create a DataLoader for training and test sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define the DNN model
class DNN(nn.Module):
    def __init__(self, dim_features):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(dim_features, 2)
        self.fc2 = nn.Linear(2, 1)


    def forward(self, x):
        #x = torch.relu(self.fc1(x))
        x1 = self.fc1(x)
        x1_tanh = torch.tanh(x1)
        x2 = self.fc2(x1_tanh)
        return x2, x1

# Create an instance of the DNN model
dim_features = X_train.shape[1]
model = DNN(dim_features)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model
num_epochs = 50
for epoch in range(num_epochs):
  running_loss = 0.0
  for inputs, targets in train_loader:
    optimizer.zero_grad()
    outputs, hidden = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
      
  epoch_loss = running_loss / len(train_loader)
  print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}")


(2981, 20)
Epoch 1/50 - Loss: 0.3780
Epoch 2/50 - Loss: 0.3679
Epoch 3/50 - Loss: 0.3593
Epoch 4/50 - Loss: 0.3516
Epoch 5/50 - Loss: 0.3439
Epoch 6/50 - Loss: 0.3361
Epoch 7/50 - Loss: 0.3298
Epoch 8/50 - Loss: 0.3223
Epoch 9/50 - Loss: 0.3178
Epoch 10/50 - Loss: 0.3131
Epoch 11/50 - Loss: 0.3082
Epoch 12/50 - Loss: 0.3033
Epoch 13/50 - Loss: 0.3005
Epoch 14/50 - Loss: 0.2970
Epoch 15/50 - Loss: 0.2927
Epoch 16/50 - Loss: 0.2905
Epoch 17/50 - Loss: 0.2887
Epoch 18/50 - Loss: 0.2872
Epoch 19/50 - Loss: 0.2852
Epoch 20/50 - Loss: 0.2835
Epoch 21/50 - Loss: 0.2815
Epoch 22/50 - Loss: 0.2797
Epoch 23/50 - Loss: 0.2798
Epoch 24/50 - Loss: 0.2780
Epoch 25/50 - Loss: 0.2785
Epoch 26/50 - Loss: 0.2772
Epoch 27/50 - Loss: 0.2766
Epoch 28/50 - Loss: 0.2754
Epoch 29/50 - Loss: 0.2760
Epoch 30/50 - Loss: 0.2755
Epoch 31/50 - Loss: 0.2752
Epoch 32/50 - Loss: 0.2743
Epoch 33/50 - Loss: 0.2755
Epoch 34/50 - Loss: 0.2742
Epoch 35/50 - Loss: 0.2746
Epoch 36/50 - Loss: 0.2736
Epoch 37/50 - Loss: 0.2734

In [666]:
# Evaluate the model
model.eval()
test_loss = 0.0
y_pred = []
hiddens = []
with torch.no_grad():
  for inputs, targets in test_loader:
    outputs, hidden = model(inputs)
    y_pred.append(outputs.detach().numpy())
    hiddens.append(hidden.detach().numpy())
    test_loss += criterion(outputs, targets).item()

print(len(test_loader))
mse = test_loss / len(test_loader)
print(f"Mean Squared Error: {mse}")
y_pred = np.concatenate(y_pred)
print(y_pred.shape)

6
Mean Squared Error: 0.07670831711341937
(2981, 1)


In [667]:
df_appl_test = df_appl[len(df_appl_train_X):].copy()
df_appl_test['y_pred'] = y_pred
df_appl_test['y_test'] = y_test
df_appl_test.loc[df_appl_test['y_pred']<0, 'y_pred'] = 0 # type: ignore
df_appl_test['y_pred_adj'] = y_pred * (len(df_appl_test)/y_pred.sum()) # type: ignore

df_appl_test['1d_profit'] = df_appl_test['log_adj_close_diff_AAPL_1'].shift(-1).fillna(0)
df_appl_test['weighted_profit'] = df_appl_test['y_pred_adj'] * df_appl_test['1d_profit']
print(np.exp(df_appl_test['1d_profit'].sum()))
print(np.exp(df_appl_test['weighted_profit'].sum()))

18.629598719868014
16.602343177809225


In [229]:
df_test['log_adj_close'][-1] - df_test['log_adj_close'][0]

1.2034801867601814

In [230]:
df_test['next_log_adj_close_diff'] = df_test['log_adj_close_diff_1'].shift(-1)

In [231]:
df_test[df_test['y_pred']>0]['next_log_adj_close_diff'].sum()

1.1481425445140117

In [232]:
len(df_test[df_test['y_pred']>0])

3041

In [233]:
len(df_test)

3057

In [234]:
df_test[df_test['y_pred']>0]['next_log_adj_close_diff'].sum()/len(df_test[df_test['y_pred']>0])

0.0003775542731055612

In [235]:
(df_test['log_adj_close'][-1] - df_test['log_adj_close'][0])/len(df_test)

0.0003936801396009753

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Assuming you have X as the input features (shape: [20000, 20]) and y as the output (shape: [20000, 1])
X_train = data_train_X.copy()
y_train = data_train_y.copy()
X_test = data_test_X.copy()
y_test = data_test_y.copy()
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the SVM model
#svm = SVR(kernel='rbf')  # 0.18
# svm = SVR(kernel='rbf', C=0.1)  #0.17 # stricter regulation
#svm = SVR(kernel='rbf', C=10)  # looser regulation 0.24
svm = SVR(kernel='linear', C=0.1) 



svm.fit(X_train, y_train)

# Make predictions
y_pred = svm.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


In [78]:
df['log_adj_close'] = np.log(df['Adj Close'])
df['last_log_adj_close'] = df['log_adj_close'].shift(1)
GCF_features = []
for n_diff_days in [1,2,4,8,16,32,64,128,256,512]:
  col_name = f'GCF_log_adj_close_diff_{n_diff_days}'
  df[col_name] = df['log_adj_close'] - df['log_adj_close'].shift(n_diff_days)
  GCF_features.append(col_name)

df_GCF = df[512:].copy()



In [79]:
df_GCF

,Open,High,Low,Close,Adj Close,Volume,log_adj_close,last_log_adj_close,GCF_log_adj_close_diff_1,GCF_log_adj_close_diff_2,GCF_log_adj_close_diff_4,GCF_log_adj_close_diff_8,GCF_log_adj_close_diff_16,GCF_log_adj_close_diff_32,GCF_log_adj_close_diff_64,GCF_log_adj_close_diff_128,GCF_log_adj_close_diff_256,GCF_log_adj_close_diff_512
Date,,,,,,,,,,,,,,,,,,
2002-09-23 00:00:00-04:00,323.000000,323.000000,323.000000,323.000000,323.000000,0,5.777652,5.774241,0.003411,0.001239,0.018751,0.020013,0.031769,0.027304,-0.000619,0.100556,0.167814,0.164889
2002-09-24 00:00:00-04:00,325.899994,325.899994,325.899994,325.899994,325.899994,0,5.786591,5.777652,0.008938,0.012350,0.016396,0.021713,0.042306,0.048729,0.003997,0.107101,0.182260,0.157891
2002-09-25 00:00:00-04:00,322.200012,322.200012,322.200012,322.200012,322.200012,0,5.775173,5.786591,-0.011418,-0.002480,-0.001241,0.017533,0.027692,0.025143,-0.006496,0.080431,0.104602,0.151155
2002-09-26 00:00:00-04:00,320.299988,320.299988,320.299988,320.299988,320.299988,3485,5.769258,5.775173,-0.005914,-0.017333,-0.004983,0.010041,0.017003,0.019547,0.000312,0.074180,0.100067,0.149582
2002-09-27 00:00:00-04:00,319.600006,321.200012,319.000000,319.700012,319.700012,858,5.767383,5.769258,-0.001875,-0.007789,-0.010269,0.008481,0.004703,0.017353,-0.002499,0.074999,0.104075,0.153525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-12 00:00:00-04:00,1959.699951,1963.500000,1951.400024,1955.300049,1955.300049,212,7.578299,7.581822,-0.003523,-0.004236,-0.005203,-0.004389,-0.000614,-0.015428,0.048892,0.090845,0.061213,0.024750
2023-06-13 00:00:00-04:00,1960.599976,1967.500000,1940.300049,1944.599976,1944.599976,343,7.572812,7.578299,-0.005487,-0.009010,0.000978,-0.017030,-0.017384,-0.023028,0.017063,0.083567,0.051223,0.038476
2023-06-14 00:00:00-04:00,1946.199951,1958.199951,1940.300049,1955.300049,1955.300049,202,7.578299,7.572812,0.005487,0.000000,-0.004236,0.001484,-0.009923,-0.017641,0.025432,0.083813,0.054332,0.034073


In [80]:
df1 = pdr.get_data_yahoo('AAPL', start='2001-01-01')[512:]
df1.merge(df_GCF[GCF_features], how='left', left_index=True, right_index=True)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,GCF_log_adj_close_diff_1,GCF_log_adj_close_diff_2,GCF_log_adj_close_diff_4,GCF_log_adj_close_diff_8,GCF_log_adj_close_diff_16,GCF_log_adj_close_diff_32,GCF_log_adj_close_diff_64,GCF_log_adj_close_diff_128,GCF_log_adj_close_diff_256,GCF_log_adj_close_diff_512
Date,,,,,,,,,,,,,,,,
2003-01-21 00:00:00-05:00,0.253750,0.257321,0.250000,0.250357,0.212789,253456000,0.001961,-0.001119,0.014945,0.009561,0.023790,0.109630,0.129183,0.118413,0.218745,0.261078
2003-01-22 00:00:00-05:00,0.249643,0.252679,0.246429,0.247857,0.210664,215140800,0.006695,0.008656,0.025339,0.017953,0.029626,0.110098,0.143550,0.126369,0.224395,0.280572
2003-01-23 00:00:00-05:00,0.250893,0.256429,0.249107,0.253036,0.215066,228256000,0.013256,0.019951,0.018832,0.027818,0.059631,0.115317,0.153282,0.118092,0.244635,0.292724
2003-01-24 00:00:00-05:00,0.254286,0.254286,0.242143,0.246429,0.209451,305468800,0.010100,0.023356,0.032012,0.037354,0.057574,0.120809,0.169155,0.129118,0.258245,0.316148
2003-01-27 00:00:00-05:00,0.244286,0.258929,0.243750,0.252321,0.214458,391406400,0.003254,0.013353,0.033304,0.048249,0.065152,0.125596,0.165677,0.166636,0.251353,0.320893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-12 00:00:00-04:00,181.270004,183.889999,180.970001,183.789993,183.789993,54274900,-0.003523,-0.004236,-0.005203,-0.004389,-0.000614,-0.015428,0.048892,0.090845,0.061213,0.024750
2023-06-13 00:00:00-04:00,182.800003,184.149994,182.440002,183.309998,183.309998,54929100,-0.005487,-0.009010,0.000978,-0.017030,-0.017384,-0.023028,0.017063,0.083567,0.051223,0.038476
2023-06-14 00:00:00-04:00,183.369995,184.389999,182.020004,183.949997,183.949997,57462900,0.005487,0.000000,-0.004236,0.001484,-0.009923,-0.017641,0.025432,0.083813,0.054332,0.034073


In [81]:
df1

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2003-01-21 00:00:00-05:00,0.253750,0.257321,0.250000,0.250357,0.212789,253456000
2003-01-22 00:00:00-05:00,0.249643,0.252679,0.246429,0.247857,0.210664,215140800
2003-01-23 00:00:00-05:00,0.250893,0.256429,0.249107,0.253036,0.215066,228256000
2003-01-24 00:00:00-05:00,0.254286,0.254286,0.242143,0.246429,0.209451,305468800
2003-01-27 00:00:00-05:00,0.244286,0.258929,0.243750,0.252321,0.214458,391406400
...,...,...,...,...,...,...
2023-06-12 00:00:00-04:00,181.270004,183.889999,180.970001,183.789993,183.789993,54274900
2023-06-13 00:00:00-04:00,182.800003,184.149994,182.440002,183.309998,183.309998,54929100
2023-06-14 00:00:00-04:00,183.369995,184.389999,182.020004,183.949997,183.949997,57462900


In [83]:

stock_names = ['AAPL', 'INTC', 'MSFT', 'T', 'DIS', 'TSLA', 'LMT', 'NVDA']
data_train_X = []
data_train_y = []
data_test_X = []
data_test_y = []
mses = []
for name in stock_names:
  start_date= "2001-01-01"
  end_date="2019-11-30"
  df = pdr.get_data_yahoo(name, start=start_date)


  df['log_adj_close'] = np.log(df['Adj Close'])
  df['last_log_adj_close'] = df['log_adj_close'].shift(1)
  df['log_volume'] = np.log(df['Volume'])
  feature_columns = GCF_features.copy()
  for n_diff_days in [1,2,4,8,16,32,64,128,256,512]:
    df[f'log_adj_close_diff_{n_diff_days}'] = df['log_adj_close'] - df['log_adj_close'].shift(n_diff_days)
    df[f'log_volume_diff_{n_diff_days}'] = df['log_volume'] - df['log_volume'].shift(n_diff_days)
    feature_columns.append(f'log_adj_close_diff_{n_diff_days}')
    feature_columns.append(f'log_volume_diff_{n_diff_days}')
    
  df['1y_future_profit'] = df['log_adj_close'].shift(-256) - df['log_adj_close']
  df = df[512:-256]
  df = df.merge(df_GCF[GCF_features], how='left', left_index=True, right_index=True)
  training_split = int(len(df) * 0.7)
  df_train = df[:training_split]
  df_test = df[training_split:]
  
  mses.append(((df_train['log_adj_close_diff_256'] - df_train['1y_future_profit'])**2).sum()/len(df_train))
  
  data_train_X.append(df_train[feature_columns].values)
  data_train_y.append(df_train['1y_future_profit'].values)
  data_test_X.append(df_test[feature_columns].values)
  data_test_y.append(df_test['1y_future_profit'].values)
  
#feature_columns

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [84]:
data_train_X = np.vstack(data_train_X)
data_test_X = np.vstack(data_test_X)
data_train_y = np.concatenate(data_train_y)
data_test_y = np.concatenate(data_test_y)

In [85]:
data_train_X.shape

(25666, 30)

In [86]:
import numpy as np

correlations = []
for column in range(data_train_X.shape[1]):
    correlation_matrix = np.corrcoef(data_train_X[:, column], data_train_y, rowvar=False)
    correlation = correlation_matrix[0, 1]
    correlations.append(correlation)
    
for i, name in enumerate(feature_columns):
  print(f'{name}: {correlations[i]}')




GCF_log_adj_close_diff_1: nan
GCF_log_adj_close_diff_2: nan
GCF_log_adj_close_diff_4: nan
GCF_log_adj_close_diff_8: nan
GCF_log_adj_close_diff_16: nan
GCF_log_adj_close_diff_32: nan
GCF_log_adj_close_diff_64: nan
GCF_log_adj_close_diff_128: nan
GCF_log_adj_close_diff_256: nan
GCF_log_adj_close_diff_512: nan
log_adj_close_diff_1: 0.004678487224153381
log_volume_diff_1: 0.0008310943090242325
log_adj_close_diff_2: 0.006858397711701181
log_volume_diff_2: 0.0014627905346109285
log_adj_close_diff_4: 0.010751487382150078
log_volume_diff_4: 0.003999029937259856
log_adj_close_diff_8: 0.014994392095298088
log_volume_diff_8: 0.004911110285570152
log_adj_close_diff_16: 0.017648116225875178
log_volume_diff_16: 0.011130443007557956
log_adj_close_diff_32: 0.01582280447248161
log_volume_diff_32: 0.026880713037469754
log_adj_close_diff_64: 0.011800257186233419
log_volume_diff_64: 0.0449141356209596
log_adj_close_diff_128: -0.026142826487253846
log_volume_diff_128: 0.05908241643912354
log_adj_close_diff

In [14]:
shuffled_index = np.random.permutation(len(data_train_X))
data_train_X = data_train_X[shuffled_index]
data_train_y = data_train_y[shuffled_index]

In [15]:
data_train_X.shape

(25659, 20)

In [16]:
data_train_y.shape

(25659,)

In [17]:
import statistics

statistics.mean(mses)

0.28918070180052136

In [25]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Assuming you have X as the input features (shape: [20000, 20]) and y as the output (shape: [20000, 1])
X_train = data_train_X.copy()
y_train = data_train_y.copy()
X_test = data_test_X.copy()
y_test = data_test_y.copy()
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the SVM model
#svm = SVR(kernel='rbf')  # 0.18
# svm = SVR(kernel='rbf', C=0.1)  #0.17 # stricter regulation
#svm = SVR(kernel='rbf', C=10)  # looser regulation 0.24
svm = SVR(kernel='linear', C=0.1) 



svm.fit(X_train, y_train)

# Make predictions
y_pred = svm.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")



Mean Squared Error: 0.18471725175706039


In [18]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


# Assuming you have X as the input features (shape: [20000, 20]) and y as the output (shape: [20000, 1])
X_train = data_train_X.copy()
y_train = data_train_y.copy()
X_test = data_test_X.copy()
y_test = data_test_y.copy()

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the SVM model
svm = SVR(kernel='rbf')  # You can choose different kernels like 'linear', 'poly', 'sigmoid', etc.
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.1, 1, 10]
}

grid_search = GridSearchCV(svm, param_grid, cv=5, verbose=2, n_jobs=2)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)


Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=  36.6s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=  37.0s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=  36.3s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=  37.9s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  27.8s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=  37.2s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  27.5s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  26.6s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  28.2s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=  26.6s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=  37.2s
[CV] END ......................C=0.1, gamma=1, k

KeyboardInterrupt: 

In [21]:
grid_search.cv_results_

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [ ]:

# Make predictions
y_pred = svm.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


In [78]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Assuming you have X as the input features (shape: [20000, 20]) and y as the output (shape: [20000, 1])
# Assuming you have X as the input features (shape: [20000, 20]) and y as the output (shape: [20000, 1])
X_train = data_train_X.copy()
y_train = data_train_y.copy()
X_test = data_test_X.copy()
y_test = data_test_y.copy()

# Split the data into training and test sets
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)

# Create a DataLoader for training and test sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define the DNN model
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(20, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create an instance of the DNN model
model = DNN()

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 300
for epoch in range(num_epochs):
  running_loss = 0.0
  for inputs, targets in train_loader:
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
      
  epoch_loss = running_loss / len(train_loader)
  print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}")

# Evaluate the model
model.eval()
test_loss = 0.0
with torch.no_grad():
  for inputs, targets in test_loader:
    outputs = model(inputs)
    test_loss += criterion(outputs, targets).item()

mse = test_loss / len(test_loader)
print(f"Mean Squared Error: {mse}")


Epoch 1/100 - Loss: 0.1134
Epoch 2/100 - Loss: 0.1093
Epoch 3/100 - Loss: 0.1068
Epoch 4/100 - Loss: 0.1035
Epoch 5/100 - Loss: 0.1003
Epoch 6/100 - Loss: 0.0967
Epoch 7/100 - Loss: 0.0936
Epoch 8/100 - Loss: 0.0907
Epoch 9/100 - Loss: 0.0879
Epoch 10/100 - Loss: 0.0857
Epoch 11/100 - Loss: 0.0828
Epoch 12/100 - Loss: 0.0807
Epoch 13/100 - Loss: 0.0782
Epoch 14/100 - Loss: 0.0773
Epoch 15/100 - Loss: 0.0754
Epoch 16/100 - Loss: 0.0736
Epoch 17/100 - Loss: 0.0724
Epoch 18/100 - Loss: 0.0713
Epoch 19/100 - Loss: 0.0706
Epoch 20/100 - Loss: 0.0691
Epoch 21/100 - Loss: 0.0681
Epoch 22/100 - Loss: 0.0669
Epoch 23/100 - Loss: 0.0666
Epoch 24/100 - Loss: 0.0654
Epoch 25/100 - Loss: 0.0647
Epoch 26/100 - Loss: 0.0635
Epoch 27/100 - Loss: 0.0636
Epoch 28/100 - Loss: 0.0624
Epoch 29/100 - Loss: 0.0619
Epoch 30/100 - Loss: 0.0618
Epoch 31/100 - Loss: 0.0608
Epoch 32/100 - Loss: 0.0602
Epoch 33/100 - Loss: 0.0600
Epoch 34/100 - Loss: 0.0594
Epoch 35/100 - Loss: 0.0591
Epoch 36/100 - Loss: 0.0588
E

In [16]:
((df['log_adj_close_diff_256'] - df['1y_future_profit'])**2).sum()/len(df)

0.3022686975386343

/tmp/ipykernel_605802/3843108136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['1y_future_profit'] = df['log_adj_close'].shift(-252) - df['log_adj_close']


In [7]:
df

,Open,High,Low,Close,Adj Close,Volume,log_adj_close,last_log_adj_close,log_volume,log_adj_close_diff_1,...,log_volume_diff_32,log_adj_close_diff_64,log_volume_diff_64,log_adj_close_diff_128,log_volume_diff_128,log_adj_close_diff_256,log_volume_diff_256,log_adj_close_diff_512,log_volume_diff_512,1y_future_profit
Date,,,,,,,,,,,,,,,,,,,,,
2002-01-14 00:00:00-05:00,0.375179,0.382143,0.373214,0.377679,0.321006,415996000,-1.136297,-1.141037,19.846186,0.004740,...,0.437625,0.175818,0.219046,-0.040764,0.317204,0.351959,-0.083697,NaN,NaN,-0.369934
2002-01-15 00:00:00-05:00,0.380714,0.388571,0.378750,0.387500,0.329353,290320800,-1.110626,-1.136297,19.486497,0.025671,...,0.147084,0.186385,0.008679,-0.045053,-0.150482,0.281555,-1.034743,NaN,NaN,-0.408001
2002-01-16 00:00:00-05:00,0.382321,0.382321,0.366071,0.371071,0.315389,566893600,-1.153948,-1.110626,20.155682,-0.043322,...,1.028125,0.144174,0.575758,-0.017177,0.360602,0.197105,-0.265662,NaN,NaN,-0.351601
2002-01-17 00:00:00-05:00,0.392143,0.406071,0.390536,0.401429,0.341192,660576000,-1.075311,-1.153948,20.308623,0.078637,...,0.776374,0.221700,1.180155,-0.002664,0.339303,0.316871,0.471225,NaN,NaN,-0.466452
2002-01-18 00:00:00-05:00,0.392857,0.403571,0.392143,0.395893,0.336486,338811200,-1.089198,-1.075311,19.640954,-0.013887,...,0.626031,0.266115,0.171066,-0.094202,-0.595857,0.291599,-0.098008,NaN,NaN,-0.458256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-09 00:00:00-04:00,181.500000,182.229996,180.630005,180.960007,180.960007,48870700,5.198276,5.196119,17.704689,0.002157,...,0.003210,0.185097,-0.096720,0.213272,-0.342409,0.185560,-0.392311,0.379375,-0.660750,NaN
2023-06-12 00:00:00-04:00,181.270004,183.889999,180.970001,183.789993,183.789993,54274900,5.213794,5.198276,17.809572,0.015518,...,0.176376,0.214591,-0.233828,0.254487,-0.176120,0.240396,-0.489735,0.400255,-0.272990,NaN
2023-06-13 00:00:00-04:00,182.800003,184.149994,182.440002,183.309998,183.309998,54929100,5.211179,5.213794,17.821554,-0.002615,...,-0.166840,0.198797,-0.430200,0.265752,-0.238460,0.232567,-0.265029,0.400292,-0.208113,NaN
